In [9]:
import psycopg2
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np

In [10]:
conn_string='host=summer22data.postgres.database.azure.com user=student_dennis dbname=daily_dennis password=dennis'

In [11]:
conn = psycopg2.connect(conn_string)
if conn.closed == 0:
    print("Connection established")

Connection established


In [12]:
fighters_query = 'SELECT * FROM fighters'
df_fighters = pd.read_sql(fighters_query, conn)
odds_query = 'SELECT * FROM odds'
df_odds = pd.read_sql(odds_query, conn)
fights_query = 'SELECT * from mma_fights'
df_fights = pd.read_sql(fights_query, conn)
conn.close()

C:\Users\AleksanderHalfstadAa\AppData\Local\Temp\ipykernel_15060\186172522.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_fighters = pd.read_sql(fighters_query, conn)
C:\Users\AleksanderHalfstadAa\AppData\Local\Temp\ipykernel_15060\186172522.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_odds = pd.read_sql(odds_query, conn)
C:\Users\AleksanderHalfstadAa\AppData\Local\Temp\ipykernel_15060\186172522.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_fights = pd.read_sql(fights_query, conn)


In [13]:
def scale_and_log_columns(dataframe, columns_to_scale, columns_to_log):
    
    transformed_df = dataframe.copy()
    scaler = StandardScaler()
    
    transformed_df[columns_to_scale] = scaler.fit_transform(transformed_df[columns_to_scale])

    transformed_df[columns_to_log] = transformed_df[columns_to_log].apply(lambda x: np.log1p(x) if x.min() > 0 else x)

    return transformed_df

Fighter Scaling

In [14]:
features_to_scale = [
    'r_avg_sig_str_landed',
    'r_avg_sig_str_landed',
    'r_avg_sig_str_pct',
    'r_avg_sub_att',
    'r_avg_td_landed',
    'r_avg_td_pct',
    'r_height_cms',
    'r_reach_cms', 
    'r_age']
features_to_log= ['r_avg_sig_str_landed', 'r_avg_sub_att', 'r_avg_td_landed']
processed_fighter =scale_and_log_columns(df_fighters, features_to_scale, features_to_log)

In [15]:
columns_to_fill_0 = [
    'r_avg_sig_str_landed',
    'r_avg_sig_str_landed',
    'r_avg_sig_str_pct',
    'r_avg_sub_att',
    'r_avg_td_landed',
    'r_avg_td_pct',
    'b_avg_sig_str_landed',
    'b_avg_sig_str_landed',
    'b_avg_sig_str_pct',
    'b_avg_sub_att',
    'b_avg_td_landed',
    'b_avg_td_pct'
]
#df_fights[columns_to_fill_0]=df_fights[columns_to_fill_0].fillna(0, inplace=True)

df_fights[columns_to_fill_0] = df_fights[columns_to_fill_0].fillna(0)

ValueError: Columns must be same length as key

Fight Scaling

In [ ]:
features_to_scale = [
    'r_avg_sig_str_landed',
    'r_avg_sig_str_landed',
    'r_avg_sig_str_pct',
    'r_avg_sub_att',
    'r_avg_td_landed',
    'r_avg_td_pct',
    'r_height_cms',
    'r_reach_cms', 
    'r_age',
    'b_avg_sig_str_landed',
    'b_avg_sig_str_landed',
    'b_avg_sig_str_pct',
    'b_avg_sub_att',
    'b_avg_td_landed',
    'b_avg_td_pct',
    'b_height_cms',
    'b_reach_cms', 
    'b_age']
features_to_log= ['r_avg_sig_str_landed', 'r_avg_sub_att', 'r_avg_td_landed',
                  'b_avg_sig_str_landed', 'b_avg_sub_att', 'b_avg_td_landed']
processed_fights =scale_and_log_columns(df_fights, features_to_scale, features_to_log)

In [ ]:
red_columns = []
blue_columns = []
for col in processed_fights.columns:
    if col.startswith('r_'):
        red_columns.append(col)
    elif col.startswith('b_'):
        blue_columns.append(col)

df_ufc_swapped = processed_fights.copy()

for r_col, b_col in zip(red_columns, blue_columns):
    df_ufc_swapped[[r_col, b_col]] = df_ufc_swapped[[b_col, r_col]]

winner_swap = {'Red': 'Blue', 'Blue': 'Red'}
df_ufc_swapped['winner'] = df_ufc_swapped['winner'].map(winner_swap)

df_ufc_combined = pd.concat([processed_fights, df_ufc_swapped], ignore_index=True)

In [ ]:
df_ufc_combined.to_csv('../CSV Files/df_ufc_masters_w_reversed.csv', index=False)
processed_fighter.to_csv('../CSV Files/unique_fighters.csv', index=False)